In [15]:
import pandas as pd
import numpy as np
import re

path = "2026_MCM_Problem_C_Data.csv"
df = pd.read_csv(path)

# 1) 把淘汰后填的 0 分当作缺失
judge_score_cols = [c for c in df.columns if re.match(r"week\d+_judge\d+_score", c)]
df[judge_score_cols] = df[judge_score_cols].replace(0, np.nan)

# 2) 每周总分（4 位评委相加）
weeks = range(1, 12)  # week1..week11
for w in weeks:
    cols = [f"week{w}_judge{j}_score" for j in range(1, 5)]
    df[f"week{w}_total"] = df[cols].sum(axis=1, min_count=1)

total_cols = [f"week{w}_total" for w in weeks]

# 3) 评委“平均周总分”（只在有效周上求均值）
df["judge_avg_per_week"] = df[total_cols].mean(axis=1, skipna=True)

# 4) 计算每周“当周参赛者内”的评委排名百分位(0最好,1最差)，用于辅助解释
rank_frames = []
for w in weeks:
    col = f"week{w}_total"
    tmp = df[["season", "celebrity_name", col]].dropna(subset=[col]).copy()
    tmp["judge_rank"] = tmp.groupby("season")[col].rank(method="average", ascending=False)
    tmp["n_week"] = tmp.groupby("season")[col].transform("count")
    tmp["judge_pct"] = (tmp["judge_rank"] - 1) / (tmp["n_week"] - 1)
    tmp["week"] = w
    rank_frames.append(tmp[["season", "celebrity_name", "week", "judge_rank", "n_week", "judge_pct"]])

ranks = pd.concat(rank_frames, ignore_index=True)
ranks["last"] = ranks["judge_rank"] == ranks["n_week"]

stats = ranks.groupby(["season", "celebrity_name"]).agg(
    weeks=("week", "count"),
    avg_weekly_pct=("judge_pct", "mean"),
    last_weeks=("last", "sum"),
).reset_index()

# 5) 季内：用 judge_avg_per_week 得到“评委名次”
df["judge_avg_rank"] = df.groupby("season")["judge_avg_per_week"].rank(method="average", ascending=False)

base = df[["season", "celebrity_name", "placement", "results", "judge_avg_rank"]].drop_duplicates()
base = base.merge(stats, on=["season", "celebrity_name"], how="left")

# 6) 冲突判定：delta = 最终名次 - 评委名次
base["delta_rank"] = base["placement"] - base["judge_avg_rank"]

season_n = (base.groupby("season")["celebrity_name"]
            .nunique()
            .reset_index(name="season_total_competitors"))

base = base.merge(season_n, on="season", how="left")

k = 4
fans_support = base[base["delta_rank"] <= -k].sort_values(["delta_rank", "season"])  # 粉丝抬上去
fans_support.to_csv('fans_support.csv', index=False, encoding="utf-8-sig")
fans_against = base[base["delta_rank"] >=  k].sort_values(["delta_rank", "season"], ascending=[False, True])  # 粉丝压下去
fans_against.to_csv('fans_against.csv', index=False, encoding="utf-8-sig")

cols = ["season","celebrity_name","season_total_competitors","placement","judge_avg_rank","delta_rank","avg_weekly_pct","last_weeks","weeks","results"]
print("=== Fans support (judges low, final high) ===")
print(fans_support[cols].to_string(index=False))
print("\n=== Fans against (judges high, final low) ===")
print(fans_against[cols].to_string(index=False))


df1 = pd.read_csv('fans_support.csv')
df2 = pd.read_csv('fans_against.csv')

# 1) 校验表头完全一致
if list(df1.columns) != list(df2.columns):
    raise ValueError(f"Headers not identical.\nfile1: {list(df1.columns)}\nfile2: {list(df2.columns)}")

# 2) 合并（上下拼接）
merged = pd.concat([df1, df2], ignore_index=True)

# 3) 可选：去掉完全重复的行
merged = merged.drop_duplicates().reset_index(drop=True)

# 4) 保存
merged.to_csv("fans_merged.csv", index=False, encoding="utf-8-sig")


=== Fans support (judges low, final high) ===
 season   celebrity_name  season_total_competitors  placement  judge_avg_rank  delta_rank  avg_weekly_pct  last_weeks  weeks            results
     27      Bobby Bones                        13          1             8.0        -7.0        0.810772           2      9          1st Place
      7  Cloris Leachman                        13          7            12.0        -5.0        0.911706           3      6  Eliminated Week 6
     17     Bill Engvall                        12          4             9.0        -5.0        0.887534           6     11 Eliminated Week 11
     19  Michael Waltrip                        13          7            12.0        -5.0        0.916319           4      8  Eliminated Week 8
     21   Alek Skarlatos                        13          3             8.0        -5.0        0.682576           2     11          3rd Place
     27      Joe Amabile                        13          6            11.0        -5.0 

In [17]:
import pandas as pd

# 文件名定义
file_fans = 'fans_merged.csv'
file_sim = 'task2_simulation_results_v3.csv'
output_file = 'result_filtered.csv'

try:
    # 1. 读取 fans_merge.csv 并提取 season 列的唯一值
    df_fans = pd.read_csv(file_fans)
    target_seasons = df_fans['season'].unique()
    
    print(f"从 {file_fans} 中提取到 {len(target_seasons)} 个相关赛季。")
    # print(target_seasons) # 如果需要查看具体是哪些赛季，可以取消注释

    # 2. 读取 task2_simulation_results_v3.csv
    df_sim = pd.read_csv(file_sim)

    # 3. 进行双重筛选：
    # 条件 A: season 列的值必须包含在 target_seasons 中
    # 条件 B: type 列的值必须是 'Final'
    filtered_df = df_sim[
        (df_sim['season'].isin(target_seasons)) & 
        (df_sim['type'] == 'Final')
    ]

    # 4. 输出结果信息
    print(f"筛选完成。在 {file_sim} 中符合条件的行数: {len(filtered_df)}")
    print("前5行预览:")
    print(filtered_df.head())

    # 5. 保存结果到新文件 (可选)
    filtered_df.to_csv(output_file, index=False)
    print(f"结果已保存为: {output_file}")

except FileNotFoundError as e:
    print(f"错误: 找不到文件 {e.filename}，请确认文件路径正确。")
except KeyError as e:
    print(f"错误: 找不到列名 {e}，请检查CSV文件中的列名拼写（如 'season' 或 'type'）是否正确。")


从 fans_merged.csv 中提取到 18 个相关赛季。
筛选完成。在 task2_simulation_results_v3.csv 中符合条件的行数: 18
前5行预览:
    season  week   type  k_elim  is_rank_identical actual_elim sim_rank_elim  \
13       2     8  Final       0               True          []            []   
33       4    10  Final       0               True          []            []   
43       5    10  Final       0               True          []            []   
63       7    10  Final       0               True          []            []   
74       8    11  Final       0               True          []            []   

   sim_pct_elim  is_elim_diff    sim_rank_winner     sim_pct_winner  \
13           []         False        Drew Lachey        Drew Lachey   
33           []         False   Apolo Anton Ohno   Apolo Anton Ohno   
43           []         False  Helio Castroneves  Helio Castroneves   
63           []         False       Brooke Burke       Brooke Burke   
74           []         False      Shawn Johnson      Shawn Johnson   

